In [8]:
import pandas as pd
import string  
from nltk.sentiment.vader import SentimentIntensityAnalyzer as NLTK_sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from nltk.sentiment import SentimentAnalyzer
from nltk import tokenize
import time
import re

# start=time.time()


# read file
filenames = ['../data/AMD_tweets.csv']
df = pd.read_csv(filenames[0])


def refined_process_tweet(tweet):
    # Remove HTML special entities (e.g. &amp;)
    tweet = re.sub(r'\&\w*;', '', tweet)
    # Convert @username to AT_USER
    tweet = re.sub('@[^\s]+', '', tweet)
    # Remove tickers
    tweet = re.sub(r'\$\w*', '', tweet)
    # To lowercase
    tweet = tweet.lower()
    # Remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet)
    # Remove hash-tags
    tweet = re.sub(r'#\w*', '', tweet)
    # Remove Punctuation and split 's, 't, 've with a space for filter
    tweet = re.sub(r'[' + string.punctuation.replace('@', '') + ']+', ' ', tweet)
    # Remove words with 2 or fewer letters
    tweet = re.sub(r'\b\w{1,2}\b', '', tweet)
    # Remove whitespace (including new line characters)
    tweet = re.sub(r'\s\s+', ' ', tweet)
    # Remove single space remaining at the front of the tweet.
    tweet = tweet.lstrip(' ')
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    tweet = ''.join(c for c in tweet if c <= '\uFFFF')
    return tweet


def simple_process_tweet(tweet):
    my_new_string = re.sub('[^ a-zA-Z0-9]', '', tweet)
    return my_new_string


def sentiment_cal(tweet):
    sia = SentimentIntensityAnalyzer()
    score= sia.polarity_scores(tweet)
    score = float(score['compound'])
    return score


def NLTK_sentiment_cal(tweet):
    scores = NLTK_sentiment.polarity_scores(tweet)
    polarity_scores = float(scores['compound'])
    return polarity_scores


def TextBlob_sentiment_cal(tweet):
    testimonial = TextBlob(tweet)
    scores = testimonial.sentiment.polarity
    polarity_scores = float(scores)
    return polarity_scores


result = pd.DataFrame()
for i, r in df.iterrows():
    r.tweet = refined_process_tweet(r.tweet)
    score =TextBlob_sentiment_cal (str(r.tweet))
    # score = NLTK_sentiment_cal(str(r.tweet))
    # score = TextBlob_sentiment_cal(str(r.tweet))
    temp = pd.DataFrame({'date': [r.Date], 'time': [r.Time], 'tweet': [r.tweet], 'sentiment': [score]})
    result = pd.concat([result, temp])


result.to_csv('../data/AMD_Sentiment.csv', encoding='utf-8', sep=',')

result.tail(10)

,date,time,tweet,sentiment
0,2016-04-06,16:49:31,interested near this could great fit,0.3875
0,2016-04-06,17:57:21,want work view our latest opening,0.5000
0,2016-04-06,18:10:59,amd product manager radeon graphics,0.0000
0,2016-04-06,18:38:12,interested near this could great fit,0.3875
0,2016-04-06,21:21:07,software development engineer amd,0.0000
0,2016-04-06,21:34:48,see our latest and click apply design engineer,0.5000
0,2016-04-06,21:48:31,see our latest and click apply soc architect,0.5000
0,2016-04-06,22:02:12,want work view our latest opening,0.5000
0,2016-04-06,22:29:29,amd physical design flow and methodology engin...,0.0000
0,2016-04-06,23:24:03,manager corporate strategy business amd,0.0000
